In [668]:
teamName = "TEAM3"
RPC_URL = 'https://node.dexalot-dev.com/ext/bc/C/rpc'

# use input prompt to collect metamask wallent address and private key
SENDER_ADDRESS = input("Enter your metamask(or other wallet) address")
PRIVATE_KEY = input("Enter Your metamask(or other wallet) private key")


In [190]:
# global imports
import requests 
from decimal import Decimal


In [132]:
# returns the current pairs
def tradingPairs():
    res = requests.get('https://api.dexalot-dev.com/api/trading/pairs')
    
    resp = {}
    for pair in res.json():
        if pair["pair"].startswith("TEAM") and teamName not in pair["pair"] :
            continue
        resp[pair["pair"]] = pair

    return resp

pairs = tradingPairs() 

teamPair = None 
for pair in pairs :
    print(pair, "->", pairs[pair])   
    if teamName in pair :
        teamPair = pair  


ALOT/AVAX -> {'pair': 'ALOT/AVAX', 'base': 'ALOT', 'quote': 'AVAX', 'basedisplaydecimals': 1, 'quotedisplaydecimals': 4, 'baseaddress': '0x264F68550136E7D0056D53385e42b1cD070A4e7c', 'quoteaddress': None, 'mintrade_amnt': '0.300000000000000000', 'maxtrade_amnt': '4000.000000000000000000', 'base_evmdecimals': 18, 'quote_evmdecimals': 18, 'auctionmode': 0, 'auctionendtime': '2022-02-22T04:12:48.606Z', 'status': 'deployed'}
ALOT/USDT.e -> {'pair': 'ALOT/USDT.e', 'base': 'ALOT', 'quote': 'USDT.e', 'basedisplaydecimals': 2, 'quotedisplaydecimals': 2, 'baseaddress': '0x264F68550136E7D0056D53385e42b1cD070A4e7c', 'quoteaddress': '0xB18e1D9F8C094e0C489F7E5739289BD86f6e40f9', 'mintrade_amnt': '20.000000000000000000', 'maxtrade_amnt': '300000.000000000000000000', 'base_evmdecimals': 18, 'quote_evmdecimals': 6, 'auctionmode': 0, 'auctionendtime': None, 'status': 'deployed'}
AVAX/USDC.e -> {'pair': 'AVAX/USDC.e', 'base': 'AVAX', 'quote': 'USDC.e', 'basedisplaydecimals': 3, 'quotedisplaydecimals': 3,

In [3]:
# FIXME: this is not complete yet
def openOrders(tradeAddress, pair) :
    res = requests.get('https://api.dexalot-dev.com/api/trading/openorders/params?traderaddress=%s&pair=%s' % (tradeAddress, pair))
    print(res.json())

res = openOrders(test, teamPair)
print(res)




NameError: name 'test' is not defined

In [287]:
from web3 import HTTPProvider, Web3
from web3.middleware import geth_poa_middleware
from eth_account import Account
from web3.middleware import construct_sign_and_send_raw_middleware

_registered_accounts = {}
def register_private_key(web3: Web3, private_key):
    assert (isinstance(web3, Web3))
    account = Account.privateKeyToAccount(private_key)

    _registered_accounts[(web3, account.address)] = account
    web3.middleware_onion.add(construct_sign_and_send_raw_middleware(account))
    web3.eth.default_account = account.address

web3 = Web3(HTTPProvider(RPC_URL))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)

register_private_key(web3, PRIVATE_KEY)



In [288]:
 # get AVAX balance for current account wallent
def getBalance(): 
    return web3.fromWei(web3.eth.get_balance(SENDER_ADDRESS), 'ether')
    
print("Account Balance", getBalance())

Account Balance 498.929885325


In [289]:
deployments = {}
def get_deployment(deployType) :
    if deployType not in deployments.keys() :
        deployments[deployType] = requests.get("https://api.dexalot-dev.com/api/trading/deploymentabi/%s" % deployType).json()
    return deployments[deployType]

def get_deployment_exchange() :
    return get_deployment("Exchange")

def get_deployment_portfolio() :
    return get_deployment("Portfolio")

def get_deployment_trade_pairs() :
    return get_deployment("TradePairs")

def get_deployment_order_books() :
    return get_deployment("Orderbooks")


In [290]:

def get_contract_exchange() :
    deploy = get_deployment_exchange()
    return web3.eth.contract(address=deploy["address"], abi=deploy["abi"]["abi"])

def get_contract_portfolio() :
    deploy = get_deployment_portfolio()
    return web3.eth.contract(address=deploy["address"], abi=deploy["abi"]["abi"])

def get_contract_order_books() :
    deploy = get_deployment_order_books()
    return web3.eth.contract(address=deploy["address"], abi=deploy["abi"]["abi"])

def get_contract_trade_pairs() :
    deploy = get_deployment_trade_pairs()
    return web3.eth.contract(address=deploy["address"], abi=deploy["abi"]["abi"])


In [669]:
from eth_utils.currency import to_wei, MIN_WEI, MAX_WEI
from eth_utils.types import is_string, is_integer
import decimal

# takes a number and converts it to any other ether unit.
def to_wei(value, unit_number, percision) :
    value = float(value)
    value = ('%%.%df' % (percision-1)) % value

    if is_integer(value) or is_string(value):
        d_value = decimal.Decimal(value=value)
    elif isinstance(value, float):
        d_value = decimal.Decimal(value=str(value))
    elif isinstance(value, decimal.Decimal):
        d_value = value
    else:
        raise TypeError("Unsupported type.  Must be one of integer, float, or string")

    s_number = str(unit_number)
 
    unit_str = "1"
    while len(unit_str) <= unit_number :
        unit_str = unit_str + "0"
    unit_value = decimal.Decimal(unit_str)
    if d_value == decimal.Decimal(0):
        return int(0)

    if d_value < 1 and "." in s_number:
        multiplier = len(s_number) - s_number.index(".") - 1
        d_value = decimal.Decimal(value=value) * 10 ** multiplier
        unit_value /= 10 ** multiplier

    result_value = decimal.Decimal(value=d_value) * unit_value

    if result_value <  MIN_WEI or result_value > MAX_WEI:
        raise ValueError("Resulting wei value must be between 1 and 2**256 - 1")

    return int(round(result_value))

# takes a number of a unit and converts it to float.
def from_wei(value, unit_number, percision) :
    if value == 0:
        return 0

    if value < MIN_WEI or value > MAX_WEI:
        raise ValueError("value must be between 1 and 2**256 - 1")

    unit_str = "1"
    while len(unit_str) <= unit_number :
        unit_str = unit_str + "0"
    unit_value = decimal.Decimal(unit_str)

    d_value = decimal.Decimal(value=value)
    result_value = d_value / unit_value

    # TODO: return decimal.Decimal instead
    return float(result_value)

def format_decimal_quote(value) :
   return to_wei(value, pairs[teamPair]["quote_evmdecimals"], pairs[teamPair]["quotedisplaydecimals"])

def format_decimal_base(value) :
   return to_wei(value, pairs[teamPair]["base_evmdecimals"], pairs[teamPair]["basedisplaydecimals"])

#human = 1.90
#expected = 1900000000000000000
#res = format_decimal_quote(human)
#if res == expected :
#    print("YES", res)
#else :
#    print("NO,", "want", expected, "got", res)    

def parse_decimal_quote(value) :
    return from_wei(value, pairs[teamPair]["quote_evmdecimals"], pairs[teamPair]["quotedisplaydecimals"])

def parse_decimal_base(value) :
    return from_wei(value, pairs[teamPair]["base_evmdecimals"], pairs[teamPair]["basedisplaydecimals"])


In [418]:

def get_symbol(tradePairId, isBase):
    contract = get_contract_exchange()
    return contract.functions.getSymbol(web3.toBytes(text=tradePairId), isBase).call()

teamSymbolName = teamPair.split("/")[0]
nativeSymbolName = teamPair.split("/")[1]

nativeSymbol = get_symbol(teamPair, False)
print(nativeSymbolName, "->", nativeSymbol)

teamSymbol = get_symbol(teamPair, True)
print(teamSymbolName, "->", teamSymbol)


AVAX -> b'AVAX\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
TEAM3 -> b'TEAM3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [419]:
# deposit tokens from wallent into portfolio 
def deposit_token(fromAddress, symbol, quantity):
    portfolio = get_contract_portfolio()

    fromAddress = Web3.toChecksumAddress(fromAddress)

    quantity = format_decimal_base(quantity)
   
    tx = portfolio.functions.depositToken(fromAddress, symbol, quantity).transact()
    print(tx)
   
    return tx


In [297]:
# Deposit some AVAX to issue buy orders.
# FIXME: this does not work yet
deposit_token(SENDER_ADDRESS, nativeSymbol, 1)

fromAddress >> 0xe01292155F805E828b2804b556Dca1a79ead7Cb2
symbol >> b'AVAX\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
quantity >> 1000000000000000000


ContractLogicError: execution reverted: P-ETNS-01

In [298]:
# Deposit some TEAM3 to issue sell orders.
deposit_token(SENDER_ADDRESS, teamSymbol, 1)

fromAddress >> 0xe01292155F805E828b2804b556Dca1a79ead7Cb2
symbol >> b'TEAM3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
quantity >> 1000000000000000000
b'\xa92\xd8N\xfa\xc8-K\xf7\x10\xdf\t9\xad\x88\xe3$*s\xbfiSj\x94\xd4\xa5\x92^\xcc\xb3UG'


HexBytes('0xa932d84efac82d4bf710df0939ad88e3242a73bf69536a94d4a5925eccb35547')

In [420]:
ORDER_BOOK_DEPTH_TOP_OF_BOOK = 1
ORDER_BOOK_DEPTH_BEST_PRICE = 2

last_buy_order_id = "" 
last_sell_order_id = "" 

# get the buy order book from the block chain
def getOrderBookBuy(trade_pair_id, req_depth) :
    global last_buy_order_id
    contract = get_contract_trade_pairs() 

    res = contract.functions.getNBuyBook(web3.toBytes(text=trade_pair_id), req_depth, 0, 0, last_buy_order_id).call()

    orders = [] 

    cur_price = res[2]
    last_buy_order_id = res[3]

    if len(res[0]) > 0 :
        for f in res[0] :
            if f != 0 :
                orders.append(f)

    if len(res[1]) > 0 :
        for f in res[1] :
            if f != 0 :
                orders.append(f)
    

    return orders, cur_price

# get the sell order book from the block chain
def getOrderBookSell(trade_pair_id, req_depth) :
    global last_sell_order_id

    contract = get_contract_trade_pairs() 

    res = contract.functions.getNSellBook(web3.toBytes(text=trade_pair_id), req_depth, 0, 0, last_sell_order_id).call()

    orders = [] 

    cur_price = res[2]
    last_sell_order_id = res[3]

    if len(res[0]) > 0 :
        for f in res[0] :
            if f != 0 :
                orders.append(f)

    if len(res[1]) > 0 :
        for f in res[1] :
            if f != 0 :
                orders.append(f)
    

    return orders, cur_price


In [663]:

ORDER_SIDE_BUY = 0 
ORDER_SIDE_SELL = 2

ORDER_TYPE_MARKET = 0
ORDER_TYPE_LIMIT = 1

# add a new order to the order book for a trade pair
def add_order(trade_pair_id, price, quantity, order_side, order_type):
    contract = get_contract_trade_pairs()

    price = format_decimal_quote(price)
    quantity = format_decimal_base(quantity)

    built_transaction = {
        "nonce": web3.eth.get_transaction_count(web3.eth.default_account),
        #'gasPrice': gas_price,
        #'gas': gas_limit,
        #'chainId': chainId,
        #'value': eth_value,
    }

    txn = contract.functions.addOrder(web3.toBytes(text=trade_pair_id), price, quantity, order_side, order_type).buildTransaction(built_transaction)
    signed_txn = web3.eth.account.sign_transaction(txn, private_key=PRIVATE_KEY)
    tx_token = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
    
    print(tx_token)

    return tx_token




In [665]:
# import pandas library
import pandas as pd
import math 
 
order_book_df = None 

# log order meta to a panda dataframe for runtime stats
def log_order_book(mid_price, spread_amount) :
    global order_book_df
    row = {'mid_price': mid_price, 'spread_amount': spread_amount}

    if order_book_df is None :
        # create the pandas DataFrame
        order_book_df = pd.DataFrame([], columns = row.keys())

    row_df = pd.DataFrame([row])
    order_book_df = pd.concat([order_book_df, row_df], ignore_index=False)
    return order_book_df

# compute the average mid price for the last N periods
def avg_mid_price(window=5) :
    avg = order_book_df['mid_price'].rolling(window).mean().tail(1).values[0]
  
    if math.isnan(avg) :
        avg = 0 

    return float(avg)    

# compute the average spread amounts for the last N periods
def avg_spread_amount(window=5) :
    avg = order_book_df['spread_amount'].rolling(window).mean().tail(1).values[0]
    
    if math.isnan(avg) :
        avg = 0 

    return float(avg)


In [666]:
import random 

# the default price used if not orders avialable to determine price
INITIAL_PRICE = 2.0

DEFAULT_QUANTITY = 1

# bid is the maximum price that a buyer is willing to pay
_, bid_price_raw = getOrderBookBuy(teamPair, ORDER_BOOK_DEPTH_TOP_OF_BOOK)
bid_price = parse_decimal_quote(bid_price_raw)

print("bid_price", bid_price, "bid_price_raw", bid_price_raw)

# ask is the minimum price that a seller is willing to accept 
_, ask_price_raw = getOrderBookSell(teamPair, ORDER_BOOK_DEPTH_TOP_OF_BOOK)
ask_price = parse_decimal_quote(ask_price_raw)

print("ask_price", ask_price, "ask_price_raw", ask_price_raw)


mid_price = None 
spread_amount = 0 
if bid_price > 0 and ask_price > 0 :
    mid_price = (bid_price + ask_price) / 2
    spread_amount = ask_price - bid_price
elif bid_price > 0 :
    mid_price = bid_price
elif ask_price > 0 :
    mid_price = ask_price    
else :
    mid_price = INITIAL_PRICE

log_order_book(mid_price, spread_amount)

avg_price = avg_mid_price()
avg_spread = avg_spread_amount()

base_price = avg_price
if base_price == 0 :
    base_price = mid_price

# TODO: this should be set to the gas amount 
min_spread_amount = 0.1

if avg_spread > 0 :
    max_spread_amount = min_spread_amount + (avg_spread / 2)
else :
    max_spread_amount = min_spread_amount + 0.01

target_spread_amount = random.uniform(min_spread_amount, max_spread_amount)

print("avg_price %.2f" % (avg_price))
print("base_price %.2f" % (base_price))
print("avg_spread %.2f" % (avg_spread))
print("min_spread_amount %.2f" % (min_spread_amount))
print("max_spread_amount %.2f" % (max_spread_amount))
print("target_spread_amount %.2f" % (target_spread_amount))

mintrade_amnt = float(pairs[teamPair]['mintrade_amnt'])
maxtrade_amnt = float(pairs[teamPair]['maxtrade_amnt'])
print("mintrade_amnt %.2f" % (mintrade_amnt))
print("maxtrade_amnt %.2f" % (maxtrade_amnt))




bid_price 2 bid_price_raw 1892000000000000000
ask_price 0 ask_price_raw 0
0   NaN
Name: mid_price, dtype: float64
avg_price 0.00
base_price 2.00
avg_spread 0.00
min_spread_amount 0.10
max_spread_amount 0.11
target_spread_amount 0.11
mintrade_amnt 0.30
maxtrade_amnt 4000.00


In [656]:
buy_price = base_price - target_spread_amount
buy_quantity = DEFAULT_QUANTITY
print("buy_price %.2f %.2f" % (buy_price, parse_decimal_quote(format_decimal_quote(buy_price))))

buy_amount = buy_price * buy_quantity

print("buy_amount", buy_amount, "buy_quantity", buy_quantity)

while buy_amount < mintrade_amnt :
    buy_quantity = buy_quantity + 1
    buy_amount = buy_price * buy_quantity
    print(" > buy_amount", buy_amount, buy_amount, "buy_quantity", buy_quantity, "buy_price")

if buy_amount > maxtrade_amnt :
    raise Exception("buy amount exceeded max")    

res = add_order(teamPair, buy_price, buy_quantity, ORDER_SIDE_BUY, ORDER_TYPE_LIMIT)
print(res)




val1 1.892
val2 1.892
unit_value 1000000000000000000 unit_str 1000000000000000000
unit_value 1000000000000000000
resultVal2 1892000000000000000.000
resultVal3 1892000000000000000.000
buy_price 1.89 19.00
buy_amount 1.892083017436054 buy_quantity 1
mintrade_amnt 0.3 orig 0.300000000000000000
val1 1.892
val2 1.892
unit_value 1000000000000000000 unit_str 1000000000000000000
unit_value 1000000000000000000
resultVal2 1892000000000000000.000
resultVal3 1892000000000000000.000
val1 1
val2 1
unit_value 1000000000000000000 unit_str 1000000000000000000
unit_value 1000000000000000000
resultVal2 1000000000000000000
resultVal3 1000000000000000000
price >> 1892000000000000000
quantity >> 1000000000000000000
b'\\]\x85\xb4\xbb\xbb\xb5iHu\xea\x15t~~\\\xe0\xfb\xe0\xde\x1a:Y\t\xac\x7f\x92K3mB\x98'
b'\\]\x85\xb4\xbb\xbb\xb5iHu\xea\x15t~~\\\xe0\xfb\xe0\xde\x1a:Y\t\xac\x7f\x92K3mB\x98'


In [667]:

sell_price = base_price + target_spread_amount
sell_quantity = DEFAULT_QUANTITY
print("sell_price %.2f %.2f" % (sell_price, parse_decimal_quote(format_decimal_quote(sell_price))))

sell_amount = sell_price * sell_quantity

print("sell_amount", sell_amount, "sell_quantity", sell_quantity)

while sell_amount < mintrade_amnt :
    sell_quantity = sell_quantity + 1
    sell_amount = sell_price * sell_quantity
    print(" > sell_amount", sell_amount, sell_amount, "sell_quantity", sell_quantity, "sell_price")

if sell_amount > maxtrade_amnt :
    raise Exception("sell amount exceeded max")    

res = add_order(teamPair, sell_price, sell_quantity, ORDER_SIDE_SELL, ORDER_TYPE_LIMIT)
print(res)

val1 2.107
val2 2.107
unit_value 1000000000000000000 unit_str 1000000000000000000
unit_value 1000000000000000000
resultVal2 2107000000000000000.000
resultVal3 2107000000000000000.000
sell_price 2.11 2.00
sell_amount 2.106802208630649 sell_quantity 1
val1 2.107
val2 2.107
unit_value 1000000000000000000 unit_str 1000000000000000000
unit_value 1000000000000000000
resultVal2 2107000000000000000.000
resultVal3 2107000000000000000.000
val1 1
val2 1
unit_value 1000000000000000000 unit_str 1000000000000000000
unit_value 1000000000000000000
resultVal2 1000000000000000000
resultVal3 1000000000000000000
price >> 2107000000000000000
quantity >> 1000000000000000000


ContractLogicError: execution reverted